## SC04 Group 08 DDW 2D Task 1

### Introduction:

#### The appearance of COVID-19 has caused widespread death and devastation economically and socially on a scale not seen since the Spanish flu in 1918. As of November 2021, a record 5.17M deaths have been reported worldwide, and with the flu continuously spreading, this number will only increase. As such our group has created a Machine Learning Algorithm to help governments calculate the total death toll exacted by the virus.

#### Our Machine Learning Algorithm will aim to predict the total deaths due to COVID-19 of any geographical area on Earth, as long as the necessary data from our determined variables are provided.


##### Assumptions: 
1. We will assume that data used to train the machine learning algorithm is not affected by the geographical region it originates from.

2. We will assume that the data taken is accurate, and not under/over-reported.

3. We will assume that data columns that have large numbers of data values missing will have little effect on the calculation of total deaths.

### Import statements for various python libraries

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math

## Data Pre-processing

#### Importing the dataset from .csv format to a pandas dataframe and displaying it. 

In [2]:
df_main = pd.read_csv(r"owid-covid-data.csv") ## 131707 rows x 65 columns


### Listing all the countries in the dataset

In [3]:
series_main_countries = df_main['location'].drop_duplicates()  ## Series that contains all the country names: 237 countries
ls_main_countries = series_main_countries.tolist() # list of the countries


### Preliminary filtering: Removing unwanted rows and columns based on preliminary filtering


1. Several rows in the countries listing do not contain any countries data, or represents a sum of data from various countries which can distort results



2. We have also observed that several columns in the dataset are missing large segments of data. We will remove these columns as they do not provide enough data for us to accurately predict or assume its effect on total deaths.


3. We will remove several other columns which do not have a numerical value that can affect the total death (Continent, iso_code etc.)


4. Finally we will remove rows which have any data missing in them. This is to ensure that the DataFrame we are working with is complete with values from all the remaining columns. As we are predicting the deaths based on the data provided by any geographical area, the classification of where the data comes from will be assumed to have little effect.



### Removal of rows with overlapping data under the countries column


In [21]:
## Removing the rows that are not even countries: 10 values
## Asia, Europe, European Union, High income, International, Low income, 
## lower middle income, North America, Upper middle income,World

non_countries = ['Asia', 'Africa', 'Europe', 'European Union', 'High income', 'International', 'Low income', 'Lower middle income', 'North America', 'Upper middle income', 'World' ]
df_main2 = df_main[df_main['location'].isin(non_countries) == False]
ls_countries_cleaned = df_main2['location'].drop_duplicates().tolist()

#display(df_main2)

### Checking how many missing data sets each specific column has.

 - Any column which has 10000 data sets missing will be removed as they will affect the building of the training model. 10000 missing data sets is too large for us to be able to reasonably assume or insert in any values that can be used to train the model accurately.

In [22]:
all_features = ['total_cases', 'new_cases','new_cases_smoothed', 'total_deaths', 'new_deaths','new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million','total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients','icu_patients_per_million', 'hosp_patients','hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'new_tests', 'total_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 'total_boosters',
       'new_vaccinations', 'new_vaccinations_smoothed',
       'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred',
       'people_fully_vaccinated_per_hundred', 'total_boosters_per_hundred',
       'new_vaccinations_smoothed_per_million', 'stringency_index',
       'population', 'population_density', 'median_age', 'aged_65_older',
       'aged_70_older', 'gdp_per_capita', 'extreme_poverty',
       'cardiovasc_death_rate', 'diabetes_prevalence', 'female_smokers',
       'male_smokers', 'handwashing_facilities', 'hospital_beds_per_thousand',
       'life_expectancy', 'human_development_index']


def check_miss_col(main_country_list, all_features):
    display_dict = {}

    for i in main_country_list:
        display_dict[i] = main_country_list[i].isnull().sum(axis=0)
        display_dict = {k: v for k, v in sorted(display_dict.items(), key=lambda item: item[1])}
                

In [23]:
features_list = ['total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients', 
        'new_tests', 'total_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated', 
        'new_vaccinations_smoothed',
       'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred',
       'new_vaccinations_smoothed_per_million', 'stringency_index',
       'population', 'population_density', 'median_age', 'aged_65_older',
       'aged_70_older', 'gdp_per_capita', 'extreme_poverty',
       'cardiovasc_death_rate', 'diabetes_prevalence', 'female_smokers',
       'male_smokers', 'hospital_beds_per_thousand',
       'life_expectancy', 'human_development_index','new_people_vaccinated_smoothed', 'new_people_vaccinated_smoothed_per_hundred']

to_remove = ['new_vaccinations', 'people_fully_vaccinated_per_hundred', 'new_vaccinations', 'hosp_patients_per_million', 'total_boosters', 
             'total_boosters_per_hundred', 'weekly_hosp_admissions', 'weekly_hosp_admissions_per_million', 
             'weekly_icu_admissions', 'weekly_icu_admissions_per_million', 'handwashing_facilities', 'iso_code', 'continent',
            'location', 'date', 'tests_units']

def remove_empty_rows(uncleaned_countries, features_list, remove_list):
    
    uncleaned_countries = uncleaned_countries.drop(remove_list, axis=1)
    for features in features_list:
        uncleaned_countries = uncleaned_countries[uncleaned_countries[features].notna()]
    return uncleaned_countries

cleaned_countries = remove_empty_rows(df_main2, features_list, to_remove)

### Data selection:

From the cleaned dataframe above with 48 variables, we use the cor function from pandas to determine which variables have the largest correlation to the total deaths that can be calculated by our Machine Learning Model.

We then filter out which variables are the best from each catergory based on their correlation score.

In [24]:
cor = cleaned_countries.corr()
cor_target = abs(cor["total_deaths"])
relevant_features = cor_target[cor_target>0.8] 
relevant_features

total_cases                0.992716
total_deaths               1.000000
total_tests                0.962401
new_tests_smoothed         0.830114
total_vaccinations         0.912164
people_vaccinated          0.926536
people_fully_vaccinated    0.889990
population                 0.975168
Name: total_deaths, dtype: float64

### Data Visualization

We then use Seaborn to visualize the various variables of data we have selected. This is to see if we can pre-emptively determine any relationships between the variables.

In [25]:
required_columns = ['total_cases', 'total_tests', 'people_vaccinated', 'population', 'total_deaths']
df_4_vars = cleaned_countries[required_columns]
df_4_vars.to_csv('df_to_check.csv')
#sns.pairplot(df_4_vars)

### Final Data-Cleaning Procedure

We then remove the rest of the unwanted columns in the dataframe that do not have a substantial effect on the total deaths.

In [26]:
relevant_features = ['total_cases', 'total_tests', 'people_vaccinated', 'population', 'total_deaths']
removing_list = []
def remove_useless_cols(cleaned_countries, relevant_features, features_list ):
    for features in features_list:
        if features not in relevant_features:
            removing_list.append(features)
    cleaned_countries = cleaned_countries.drop(removing_list, axis=1)
    return cleaned_countries
cleaned_data = remove_useless_cols(cleaned_countries, relevant_features, features_list) 
#display(cleaned_data)

### Code from Cohort class necessary for Multiple Linear Regression¶

The following cell contains all the code necessary for training and testing the Multiple Linear Regression machine learning model. The code is closely referred to from the week 9 Cohort materials.


In [27]:
def normalize_z(dfin):
    dfout = (dfin-dfin.mean(axis=0))/dfin.std(axis=0)
    return dfout

def get_features_targets(df, feature_names, target_names):
    df_feature = df.loc[:,feature_names]
    df_target = df.loc[:,target_names]
    return df_feature, df_target

def prepare_feature(df_feature):
    cols = len(df_feature.columns)
    #convert df to numpy array for feature matrix
    feature = df_feature.to_numpy().reshape(-1,cols)
    #to get the no of rows
    m=df_feature.shape[0]
    # to create a vector of ones, m rows
    array1=np.ones((m,1))
    #add the column one with the feature matrix
    x=np.concatenate((array1,feature),axis=1)
    
    return x

def prepare_target(df_target):
    cols = len(df_target.columns)
    #convert df to numpy array for target vector
    target = df_target.to_numpy().reshape(-1,cols)
    return target

def predict(df_feature, beta):
    norm_feature = normalize_z(df_feature)
    X=prepare_feature(norm_feature)
    return predict_norm(X,beta)

def predict_norm(X, beta):
    return np.matmul(X,beta)

def split_data(df_feature, df_target, random_state=None, test_size=0.5):
    indices = df_feature.index
    if random_state !=None:
        np.random.seed(random_state)
    num_rows=len(indices)
    k=int(test_size*num_rows)
    test_indices=np.random.choice(indices,k,replace=False)
    
    indices=set(indices)
    test_indices=set(test_indices)
    train_indices=indices-test_indices
    
    df_feature_train=df_feature.loc[train_indices,:]
    df_feature_test=df_feature.loc[test_indices,:]
    df_target_train=df_target.loc[train_indices,:]
    df_target_test=df_target.loc[test_indices,:]
    return df_feature_train, df_feature_test, df_target_train, df_target_test
  
def r2_score(y, ypred):
    
    mean_y=np.mean(y)
    
    ss_res=np.sum((y-ypred)**2)
    ss_tot=np.sum((y-mean_y)**2)
    r_square=1-(ss_res/ss_tot)
    print(len(ypred))
    return r_square

def adjusted_r2(y,r2,beta):

    adj = 1-((1-r2)*(len(y)-1)/(len(y)-len(beta)-1))
    return adj

def mean_squared_error(target, pred):
    n=target.shape[0]
    
    return (1/n)*np.sum((target-pred)**2)


def root_mean_squared_error(target, pred):
    n=target.shape[0]
    
    return math.sqrt((1/n)*np.sum((target-pred)**2))

def mean_average_square(target, pred):
    n=target.shape[0]
    
    return (1/n)*np.sum(abs(target-pred))

def compute_cost(X, y, beta):
    J = 0
    m=X.shape[0]
    yp=np.matmul(X,beta)
    error=yp-y
    J=(1/(2*m)) *np.matmul(error.T,error)
    #print(J)

    J = J[0][0]
    return J


def gradient_descent(X, y, beta, alpha, num_iters):
    m=len(y)
    J_storage=np.zeros(num_iters)
    for n in range(num_iters):
        #calculate yp,predicted value using the old beta
        yp=np.matmul(X,beta)
        #calculate the error yp-y
        error=yp-y
        #calculate the delta for beta
        delta=np.matmul(X.T,error)
        #caluclate the new beta
        beta=beta-(alpha/m)*delta
        #calculate J for the new beta
        J_storage[n]=compute_cost(X,y,beta)
    return beta, J_storage


def gradient_descent(X, y, beta, alpha, num_iters):
    m=len(y)
    J_storage=np.zeros(num_iters)
    for n in range(num_iters):
        #calculate yp,predicted value using the old beta
        yp=np.matmul(X,beta)
        #calculate the error yp-y
        error=yp-y
        #calculate the delta for beta
        delta=np.matmul(X.T,error)
        #caluclate the new beta
        beta=beta-(alpha/m)*delta
        #calculate J for the new beta
        J_storage[n]=compute_cost(X,y,beta)
    return beta, J_storage

### Method for Evaluation of Machine Learning Model

In order to validate our Machine learning Model, we have determined to use the following methods:

1. Adjusted R-Squared: We have chosen Adjusted R-Squared as one of our methods of evaluation primarily because it ensures that only variables that are useful to the Machine Learning Algorithm are used in the model. Any excess variables which do not have a meaningful impact on the model will penalize the R-Squared Value and thus prevent Over-fitting. 

2. Root-Mean-Squared-Error: We have also chosen Root-Mean-Squared-Error as another method of evaluation as it can help us accurately determine the amount of error in between the predicted and actual values from the Machine Learning Model.




### RMSE Algorithm

In [11]:
import statistics

def rmse_score(df_feature, df_target, beta, alpha, num_iters, random_state=100, test_size=0.5):
    index = df_feature.index
    index = list(index)
    
    rmse_scores = []
    
    df_features_train, df_features_test, df_target_train, df_target_test = split_data(df_feature, df_target, random_state, test_size)
    
    X = df_features_train
    X = prepare_feature(X)
    y = prepare_target(df_target_train)
    
    beta, J = gradient_descent(X, y, beta, alpha, num_iters)
    pred = predict(df_features_test, beta)
    target = prepare_target(df_target_test)
    rmse = np.sqrt(mean_squared_error(target, pred).item(0))
    rmse_scores.append(rmse)
    
    return statistics.mean(rmse_scores)

### Data-splitting and Adjusted R-Squared Calculation

In [12]:
model_features = ['total_cases', 'people_vaccinated', 'population', 'total_tests']

df_features, df_target = get_features_targets(cleaned_data, model_features, ['total_deaths'])
df_features = normalize_z(df_features)
df_target = normalize_z(df_target)
n = len(model_features)
rmse = rmse_score(df_features, df_target, np.ones((n+1,1)), 0.01, 1500, test_size=0.3)
df_features_train, df_features_test, df_target_train, df_target_test = split_data(df_features, df_target, 100, 0.3)


num_cols = len(model_features)+1
df_features_train_z = normalize_z(df_features_train)
X = prepare_feature(df_features_train_z)
target = prepare_target(df_target_train)
iterations = 3000
alpha = 0.01

beta = np.zeros((num_cols,1))
beta, J_storage = gradient_descent(X, target, beta, alpha, iterations)


pred = predict(df_features_test, beta)
target = prepare_target(df_target_test)
r2 = r2_score(target, pred)

adj_r2 = adjusted_r2(target,r2,beta)

print("Adjusted r2 value: ", adj_r2)
print('RMSE Value:', rmse)

1217
Adjusted r2 value:  0.990253423841115
RMSE Value: 0.10293399777896213


### Analysis of Data:

The adjusted R-Squared value and RMSE Values all indicate that our Machine Learning Model is relatively accurate and the results fit the model extremely well. However, there are concerns that the model may be overfitted due to the extremely high correlation that is not so commonly found.


To double-check our data and results, we have thus decided to carry out a set of Linear Regressions between the Total Deaths and the Variables we have selected.

In [13]:
def r2_score_train(df_feature, df_target, beta, alpha, num_iters, random_state=100, test_size=0.5):
    df_features_train, df_features_test, df_target_train, df_target_test = split_data(df_feature, df_target, random_state, test_size)
    df_features_train_z = normalize_z(df_features_train)
    X = prepare_feature(df_features_train_z)
    target = prepare_target(df_target_train)
    beta, J = gradient_descent(X,target,beta,alpha,num_iters)
    pred = predict(df_features_test,beta)
    target = df_target_test.to_numpy()
    r2 = r2_score(target,pred)

    return r2,df_target_test,pred

df_features, df_target = get_features_targets(cleaned_countries, ['total_cases'], ['total_deaths'])
df_features = normalize_z(df_features)
n = 1
r2,df_target_test,pred = r2_score_train(df_features, df_target, np.ones((n+1,1)), 0.01, 1500, test_size=0.3)
print("r2 value of all features",r2)


for column in model_features:
    df_features, df_target = get_features_targets(cleaned_countries, [column], ['total_deaths'])
    df_features = normalize_z(df_features)
    n = 1
    r2,df_target_test,pred = r2_score_train(df_features, df_target, np.ones((n+1,1)), 0.01, 1500, test_size=0.3)
    print("r2 value of" ,str(column),r2)
    #plt.scatter(df_features_test[column],df_target_test)
    #plt.scatter(df_features_test[column],pred)
    #plt.show()
    

1217
r2 value of all features 0.9859587777228399
1217
r2 value of total_cases 0.9859587777228399
1217
r2 value of people_vaccinated 0.8811938410264417
1217
r2 value of population 0.9628907388696735
1217
r2 value of total_tests 0.9165718091414302


In [14]:
 #plt.plot(J_storage)

### Final Analysis and Conclusions:

Reviewing the Results, we have determined that the individual variables all have a very large correlation with the total deaths itself. This justifies the large R-Squared Values and Low RMSE Values we have obtained and is not a by-product of Over-fitting.

### Further Improvements:

To further improve the Model without Over-fitting, we will now add variables to our model and use the Adjusted R-Square and RMSE as metrics to determine if the variable added produces a more accurate model. We have determined that ICU Patients are the only additional unique variable within 0.6 correlation score that can be added.

In [15]:
model_features = ['total_cases', 'people_vaccinated', 'population', 'total_tests', 'icu_patients']


df_features, df_target = get_features_targets(cleaned_countries, model_features, ['total_deaths'])
df_features = normalize_z(df_features)
df_target = normalize_z(df_target)
n = len(model_features)
rmse = rmse_score(df_features, df_target, np.ones((n+1,1)), 0.01, 1500, test_size=0.3)
df_features_train, df_features_test, df_target_train, df_target_test = split_data(df_features, df_target, 100, 0.3)


num_cols = len(model_features)+1
df_features_train_z = normalize_z(df_features_train)
X = prepare_feature(df_features_train_z)
target = prepare_target(df_target_train)
iterations = 3000
alpha = 0.01

beta = np.zeros((num_cols,1))
beta, J_storage = gradient_descent(X, target, beta, alpha, iterations)


pred = predict(df_features_test, beta)
target = prepare_target(df_target_test)
r2 = r2_score(target, pred)

adj_r2 = adjusted_r2(target,r2,beta)

print("Adjusted r2 value: ", adj_r2)
print('RMSE Value:', rmse)


1217
Adjusted r2 value:  0.9905207814865252
RMSE Value: 0.10962301960392118


#### Analysis of Result:

While the Adjusted R-Squared value has increased, the RMSE value decreases and thus we cannot make a clear decision on whether the ICU_Patients has a positive effect on the model.

In [16]:
model_features = ['total_cases', 'people_vaccinated', 'population', 'total_tests']


df_features, df_target = get_features_targets(cleaned_countries, model_features, ['total_deaths'])
df_features = normalize_z(df_features)
df_target = normalize_z(df_target)
n = len(model_features)
rmse = rmse_score(df_features, df_target, np.ones((n+1,1)), 0.01, 1500, test_size=0.3)
df_features_train, df_features_test, df_target_train, df_target_test = split_data(df_features, df_target, 100, 0.3)


num_cols = len(model_features)+1
df_features_train_z = normalize_z(df_features_train)
X = prepare_feature(df_features_train_z)
target = prepare_target(df_target_train)
iterations = 3000
alpha = 0.01

beta = np.zeros((num_cols,1))
beta, J_storage = gradient_descent(X, target, beta, alpha, iterations)


pred = predict(df_features_test, beta)
target = prepare_target(df_target_test)
r2 = r2_score(target, pred)

adj_r2 = adjusted_r2(target,r2,beta)

print("Adjusted r2 value: ", adj_r2)
print('RMSE Value:', rmse)

1217
Adjusted r2 value:  0.990253423841115
RMSE Value: 0.10293399777896213


## Final Analysis and Conclusion

The current Iteration of the Model is quite optimal. Any additional variables we can add to the model have a lower correlation value with total deaths and will likely reduce the performance of the model. Existing variables that have a high correlation with total deaths will overlap directly with the main variables we have chosen and definitely result in over-fitting of the model. 

### References

https://www.statisticshowto.com/probability-and-statistics/statistics-definitions/adjusted-r2/

https://www.statisticshowto.com/probability-and-statistics/regression-analysis/rmse-root-mean-square-error/

https://github.com/owid/covid-19-data/tree/master/public/data


